# Customer segmentation
_Jayeeta Ghosh (Aug 2018)_

____
### Create Transaction Dataset
 Main goal is to create customer level table that contains information about 
1. Customers
2. Purchase 
3. Products 
4. Discounts
5. Promotions
6. Demographics

### Workflow: 
Define time window to capture transaction and customer information: <br />
2 months between '2017-01-29' and '2017-03-30 - plan is to go atleast one fiscal year

1. cust_txn: This is a large merged table with orderline, order header, discount, product, and promotion. cust_txn2: Add a few extra columns like days into promotion, number of days a promotion runs, whether marked down or regular priced purchase etc<br />
2. cust_txn_agg: Bring a few aggregated columns like basket size, basket price etc<br />
 2a. EDA on cust_txn_agg
3. cust_agg: Customer level aggregated table with columns that makes sense based on eda on cust_txn_agg
4. cust_dem: Add Merkle demographic data with customer table<br />
 4a. EDA on cust_dem
5. Build clustering model


In [2]:
import sys, os, time
sys.path.append('/mnt/Opaisle/resources/python/')
sys.path.append('/home/ubuntu/.local/lib/python3.4/site-packages')
import helper_functions as gp
import pandas as pd

In [3]:
# set display right
pd.set_option('display.width', 4000)
pd.set_option('max_colwidth', 4000)
pd.set_option('max_rows', 100)
pd.set_option('max_columns', 200)

### 1. Create Table: cust_trn 

Join ods_orderline_t, cust_ord_txn_xref_t, ods_orderline_discounts_t, ods_product_t, and ods_promotion_t.

I used orderline the primary table and orderheader (cust_ord_txn_xref_t) to grab transactions that were ordered and the right dates and shipped. Orderheader table is great because there are two separate date columns: demand_date and ship_data so you don’t have to worry about order status messing with the dates.


### Updated on 8/2/18 after receiving Caiti's feedback


In [4]:
query_list = [
    """
    IF OBJECT_ID('dbo.cust_trn', 'U') IS NOT NULL
    DROP TABLE dbo.cust_trn
    """
]
gp.dmno_execute(query_list, connection='azure', result_set=False)

In [7]:

query_list = [
    """
    CREATE TABLE dbo.cust_trn
    --WITH(DISTRIBUTION=HASH(customer_key)) AS SELECT TOP 100  
    WITH(DISTRIBUTION=HASH(customer_key)) AS SELECT
        a.customer_key,
        a.transaction_num, 
        a.transaction_date,
        a.line_num,
        a.order_status, 
        a.product_key,
        a.item_qty, 
        a.sales_amt,
        a.on_sale_flag,
        a.tot_prd_cst_amt,
        
        b.discount_reason_cd,
        b.discount_amt,
        b.discount_seq_nbr,
        b.discount_level,
        b.mkt_pmotn_cd,
        b.promo_cd_key,
        
        c.prod_desc,
        --c.mdse_corp_id,
        --c.mdse_corp_desc,
        --c.mdse_comp_id,
        --c.mdse_comp_desc,
        c.mdse_div_id,
        c.mdse_div_desc,
        c.mdse_dept_id,
        c.mdse_dept_desc,
        c.mdse_class_id,
        c.mdse_class_desc,
        c.dept_sty_desc,
        --c.style_cd,
        --c.color_cd,
        --c.market_code,
        --c.style_color_cd,
        --c.size_model_cd,
        c.season_desc,
        c.season_cd,

        e.promo_code,
        e.promo_desc,
        e.promo_type_code,
        e.eff_date,
        e.end_date
                        
    FROM 
        dbo.ods_orderline_t AS a
    
    INNER JOIN 
    (SELECT  * 
    --transaction_type_cd, order_num
        --CASE
            --WHEN order_status = 'O' then order_num
            --WHEN order_status = 'R' then transaction_num END as trans_num


        FROM dbo.cust_ord_txn_xref_t
          
          WHERE 
          ( 
          -- between '2017-01-29' and '2018-02-03'
              (order_status = 'O' 
                  AND CONVERT(date, CAST(demand_date AS Date), 112) between '2017-01-29' and '2017-03-30')
              OR    
              (order_status = 'R' 
                  AND CONVERT(date, CAST(ship_date AS Date), 112) between '2017-01-29' and '2017-03-30'))
          AND country = 'US'
          AND brand = 'BR') as d
    
    ON a.transaction_num = d.transaction_num AND a.customer_key = d.customer_key 
    AND a.brand = d.brand AND a.country = d.country AND a.order_status = d.order_status
    
    LEFT JOIN
        dbo.ods_orderline_discounts_t as b
    ON (a.transaction_num = b.transaction_num AND a.line_num = b.line_num)
    
    LEFT JOIN 
        dbo.ods_product_t as c
    ON b.product_key = c.product_key

    LEFT JOIN
        dbo.Ods_promotion_t as e
    ON b.promo_cd_key = e.promo_cd_key
   
    WHERE 
        
        a.country = 'US'
        AND a.brand = 'BR'
        AND (a.order_status = 'R' OR a.order_status = 'O')
        --AND a.transaction_date between '2017-01-29' and '2017-03-30'
        
    """
]


In [8]:
%%time
gp.dmno_execute(query_list, connection='azure', result_set=False)
# it took 2min 38s for 2 months

CPU times: user 174 ms, sys: 43.2 ms, total: 217 ms
Wall time: 3min 49s


In [9]:
q = [
    """
    SELECT
       count(*)
          
    FROM 
        dbo.cust_trn
     """
]
r = gp.dmno_execute(q, connection='azure')
print(r)

COMPLETED! 0.32 seconds.
           
0  15390029


Jan 1-3 2018: 332,604

Jan 2018    : 3,639,272

Feb-Mar 2017: 7,686,468, after another date filter now its 7,358,704

same time window, ran query on 9/4/2018 gave 15,390,029 records 

#### Create a few extra columns

**TODO: flag_return**

In [10]:
q = [
    """
    alter table cust_trn add daysinto_promo numeric, num_promodays numeric;

    """
]
gp.dmno_execute(q, connection='azure', result_set=False)

In [11]:
q = [
    """
    update cust_trn set 
        daysinto_promo = DATEDIFF(DAY,eff_date,end_date),
        num_promodays = DATEDIFF(DAY,eff_date,transaction_date)
    """
]
gp.dmno_execute(q, connection='azure', result_set=False)

In [12]:
q = [
    """
    SELECT 
        TOP 100 *
    FROM 
         dbo.cust_trn
    """
]
df = gp.dmno_execute(q, connection='azure')

COMPLETED! 1.27 seconds.


In [13]:
df.head()

,customer_key,transaction_num,transaction_date,line_num,order_status,product_key,item_qty,sales_amt,on_sale_flag,tot_prd_cst_amt,discount_reason_cd,discount_amt,discount_seq_nbr,discount_level,mkt_pmotn_cd,promo_cd_key,prod_desc,mdse_div_id,mdse_div_desc,mdse_dept_id,mdse_dept_desc,mdse_class_id,mdse_class_desc,dept_sty_desc,season_desc,season_cd,promo_code,promo_desc,promo_type_code,eff_date,end_date,daysinto_promo,num_promodays
0,155220890,2214151975,2017-02-05,1,R,3683812290,1,59.10,N,26.09,None,NaN,NaN,None,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN
1,104160508,2121296315,2017-02-26,6,R,4193402618,1,33.48,N,87.83,None,NaN,NaN,None,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN
2,26707657,2132861350,2017-03-11,1,R,895399,1,0.00,N,0.00,None,NaN,NaN,None,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN
3,164856243,2175258241,2017-03-17,1,R,5052069180,1,68.60,N,20.36,None,NaN,NaN,None,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN
4,9380739,2130326616,2017-03-16,1,R,895399,1,0.00,N,0.00,None,NaN,NaN,None,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN


#### Create a new table with instore_date and markdown_date
##### Do not worry right now

In [14]:
# query_list = [
#     """
#     IF OBJECT_ID('dbo.cust_trn_md', 'U') IS NOT NULL
#     DROP TABLE dbo.cust_trn_md
#     """
# ]
# gp.dmno_execute(query_list, connection='azure', result_set=False)

In [15]:
# q = [
#     """
#     CREATE TABLE dbo.cust_trn_md
#     WITH(DISTRIBUTION=HASH(customer_key)) AS 
#     with data as
#     (
#     SELECT 
#         customer_key,
#         transaction_num,
#         transaction_date,
#         line_num,
#         product_key,
#         order_status,
#         season_desc, 
#         LEFT(season_desc, 1) as instore_m,
#         CASE  
#             WHEN LEFT(season_desc, 1) = 'J' THEN 1
#             WHEN LEFT(season_desc, 1) = 'F' THEN 2
#             WHEN LEFT(season_desc, 1) = 'M' THEN 3
#             WHEN LEFT(season_desc, 1) = 'A' THEN 4
#             WHEN LEFT(season_desc, 1) = 'Y' THEN 5
#             WHEN LEFT(season_desc, 1) = 'U' THEN 6
#             WHEN LEFT(season_desc, 1) = 'L' THEN 7
#             WHEN LEFT(season_desc, 1) = 'G' THEN 8
#             WHEN LEFT(season_desc, 1) = 'S' THEN 9
#             WHEN LEFT(season_desc, 1) = 'O' THEN 10
#             WHEN LEFT(season_desc, 1) = 'N' THEN 11
#             WHEN LEFT(season_desc, 1) = 'D' THEN 12
#             ELSE 3 --LEFT(season_desc, 1)
#         END as instore_month,
#         CAST(SUBSTRING(season_desc, 2,2) as int) as markdown_mm,
#         CAST(RIGHT(season_desc, 4) as int) as year
#     FROM 
#          dbo.cust_trn
#     WHERE season_desc is not null 
#     AND datalength(season_desc) = 8
#     and upper(season_desc) <> 'F1A 2007'
#    )
   
#    select customer_key, transaction_num, transaction_date, line_num, product_key, order_status, 
#        season_desc, instore_m, instore_month, year, 
#        datefromparts(year, instore_month, 1) as instore_date,
#        markdown_mm,
#        datefromparts(year, markdown_mm, 1) as markdown_date
#    from data d
            
#     """
# ]
# r = gp.dmno_execute(q, connection='azure', result_set=False)


In [16]:
# q = [
#     """
#     SELECT
#        count(*)
          
#     FROM 
#         dbo.cust_trn_md
#      """
# ]
# r = gp.dmno_execute(q, connection='azure')
# print(r)

In [17]:
# q = [
#     """
#     SELECT 
#         TOP 100 *
#     FROM 
#          dbo.cust_trn_md
#     """
# ]
# df = gp.dmno_execute(q, connection='azure')

In [18]:
# df.head()

### 2. Join this new table with the original table to create a new transaction table - cust_trn2
##### Do not worry right now

In [19]:
# query_list = [
#     """
#     IF OBJECT_ID('dbo.cust_trn2', 'U') IS NOT NULL
#     DROP TABLE dbo.cust_trn2
#     """
# ]
# gp.dmno_execute(query_list, connection='azure', result_set=False)

In [20]:

# query_list = [
#     """
#     CREATE TABLE dbo.cust_trn2
#     --WITH(DISTRIBUTION=HASH(customer_key)) AS SELECT TOP 100  
#     WITH(DISTRIBUTION=HASH(customer_key)) AS SELECT
#         a.*,

#         b.instore_date as instore_date2,
#         b.markdown_date as markdown_date2,
#         CASE  
#             WHEN a.transaction_date between b.instore_date and b.markdown_date THEN 1 --regular
            
#             WHEN a.transaction_date > b.markdown_date THEN 2 -- markdown
#             WHEN a.transaction_date < b.instore_date THEN 3 -- weird
#             ELSE 0 -- missing
#         END as regular_markdown,
#         CASE  
#             WHEN a.transaction_date between b.instore_date and b.markdown_date THEN 1 ELSE 0
#         END as regular_priced,
#         CASE  
#             WHEN a.transaction_date > b.markdown_date THEN 1 ELSE 0
#         END as markdown_priced,
#         CASE  
#             WHEN a.transaction_date < b.instore_date THEN 1 ELSE 0
#         END as before_instore
        
                        
#     FROM 
#         dbo.cust_trn AS a
    
#     LEFT JOIN
#         dbo.cust_trn_md as b
#     ON (a.customer_key = b.customer_key 
#         AND a.transaction_num = b.transaction_num 
#         AND a.line_num = b.line_num)   
         
#     """
# ]
# gp.dmno_execute(query_list, connection='azure', result_set=False)

In [21]:
# --AND a.transaction_date = b.transaction_date
#         --AND a.line_num = b.line_num
#         --AND a.product_key = b.product_key
#         --AND a.order_status = b.order_status

In [22]:
# q = [
#     """
#     SELECT
#        count(*)
          
#     FROM 
#         dbo.cust_trn2
#      """
# ]
# r = gp.dmno_execute(q, connection='azure')
# print(r)
# # both not right
# # inner join: 11,106,903
# # left join: 13,495,619

In [23]:
# q = [
#     """
#     SELECT 
#         TOP 100 
#         customer_key, transaction_date, instore_date2, markdown_date2, regular_markdown, regular_priced,
#         markdown_priced, before_instore
#     FROM 
#          dbo.cust_trn2
#     """
# ]
# df = gp.dmno_execute(q, connection='azure')

In [24]:
# df.head(10)

#### Lets create customer and trasaction level aggregated temporary table

In [ ]:
# For now lets not worry about markdown/regular priced items

In [25]:
query_list = [
    """
    IF OBJECT_ID('dbo.cust_trn_aggtemp', 'U') IS NOT NULL
    DROP TABLE dbo.cust_trn_aggtemp
    """
]
gp.dmno_execute(query_list, connection='azure', result_set=False)

In [26]:
q = [
    """
    CREATE TABLE dbo.cust_trn_aggtemp
    WITH(DISTRIBUTION=HASH(customer_key)) AS 
    SELECT 
        t.customer_key,
        t.transaction_num,
            
       SUM(item_qty) as basket_size,
       SUM(sales_amt) as basket_price
    FROM 
        --dbo.cust_trn2 as t
        dbo.cust_trn as t
    GROUP BY customer_key, transaction_num
    
    
    """
]
r = gp.dmno_execute(q, connection='azure', result_set=False)


In [27]:
q = [
    """
    SELECT
       count(*)
          
    FROM 
        dbo.cust_trn_aggtemp
     """
]
r = gp.dmno_execute(q, connection='azure')
print(r)
# 2,383,792

COMPLETED! 0.35 seconds.
          
0  2384779


In [28]:
q = [
    """
    SELECT 
        TOP 100 *
    FROM 
         dbo.cust_trn_aggtemp
    """
]
df = gp.dmno_execute(q, connection='azure')

COMPLETED! 0.21 seconds.


In [29]:
df.head()

,customer_key,transaction_num,basket_size,basket_price
0,5555598,2128012245,12,319.04
1,67166290,2138923218,12,633.60
2,87505191,2132591765,-4,-352.00
3,260020130,2137489474,2,93.60
4,164578235,2115085985,4,312.00


In [30]:
query_list = [
    """
    IF OBJECT_ID('dbo.cust_trn_agg', 'U') IS NOT NULL
    DROP TABLE dbo.cust_trn_agg
    """
]
gp.dmno_execute(query_list, connection='azure', result_set=False)

In [31]:
q = [
    """
    CREATE TABLE dbo.cust_trn_agg
    WITH(DISTRIBUTION=HASH(customer_key)) AS 
    SELECT 
        a.* ,
        b.basket_size,
        b.basket_price
        
    FROM 
        --dbo.cust_trn2 as a
        dbo.cust_trn as a
    LEFT JOIN
        dbo.cust_trn_aggtemp as b
     
    ON (a.customer_key = b.customer_key AND a.transaction_num = b.transaction_num)
    
    
    """
]
r = gp.dmno_execute(q, connection='azure', result_set=False)


In [32]:
q = [
    """
    SELECT 
        COUNT(*)
    FROM 
         dbo.cust_trn_agg
    """
]
df = gp.dmno_execute(q, connection='azure')
print(df)

COMPLETED! 0.25 seconds.
           
0  15390029


### 3. Lets create Customer level aggregated table: cust_agg

In [33]:
query_list = [
    """
    IF OBJECT_ID('dbo.cust_agg', 'U') IS NOT NULL
    DROP TABLE dbo.cust_agg
    """
]
gp.dmno_execute(query_list, connection='azure', result_set=False)

In [34]:
q = [
    """
    CREATE TABLE dbo.cust_agg
    --WITH(DISTRIBUTION=HASH(customer_key)) AS SELECT TOP 100  
    WITH(DISTRIBUTION=HASH(customer_key)) AS 
    SELECT 
       customer_key, 
       AVG(basket_size) as avg_basket_size,
       AVG(basket_price) as avg_basket_price,
       COUNT(DISTINCT transaction_num) as count_num_txn,
       COUNT(DISTINCT product_key) as count_num_product,
       SUM(sales_amt) as total_sales_amt,
      
       COUNT(item_qty) as count_num_items,  
       MIN(CASE WHEN sales_amt > 0 THEN sales_amt ELSE 0 END) as min_sales_amt,
       SUM(tot_prd_cst_amt) as total_cost_amt,
       SUM(sales_amt - tot_prd_cst_amt) as total_diff_sale_cost,
       SUM(discount_amt) as total_discount_amt,
       

       SUM(CASE WHEN sales_amt < 0 THEN 1 ELSE 0 END) count_num_return,
    
       SUM(CASE WHEN order_status = 'O' THEN 1 ELSE 0 END) as count_online_ord,
       SUM(CASE WHEN order_status = 'R' THEN 1 ELSE 0 END) as count_retail_ord,
      
       
       --SUM(CASE WHEN discount_level = '0' THEN 1 ELSE 0 END) as discount_level_0,
       --SUM(CASE WHEN discount_level = '1' THEN 1 ELSE 0 END) as discount_level_1,
       --SUM(CASE WHEN discount_level = '2' THEN 1 ELSE 0 END) as discount_level_2,
       --SUM(CASE WHEN discount_level = '3' THEN 1 ELSE 0 END) as discount_level_3,
       
       SUM(CASE WHEN on_sale_flag = 'Y' THEN 1 ELSE 0 END) as count_onsale_flag,
       COUNT(DISTINCT promo_code) as count_promo_code,
       AVG(DATEDIFF(DAY,eff_date,transaction_date)) as avg_daysinto_promo
       
       --SUM(regular_priced) as count_regular_priced,
       
       --SUM(markdown_priced) as count_markdown_priced
       
       
    FROM 
        dbo.cust_trn_agg
    GROUP BY customer_key

    
    """
]
r = gp.dmno_execute(q, connection='azure', result_set=False)


In [35]:
q = [
    """
    SELECT 
        TOP 100 *
    FROM 
         dbo.cust_agg
    """
]
df = gp.dmno_execute(q, connection='azure')

COMPLETED! 0.25 seconds.


In [36]:
df.head()

,customer_key,avg_basket_size,avg_basket_price,count_num_txn,count_num_product,total_sales_amt,count_num_items,min_sales_amt,total_cost_amt,total_diff_sale_cost,total_discount_amt,count_num_return,count_online_ord,count_retail_ord,count_onsale_flag,count_promo_code,avg_daysinto_promo
0,73060153,4,93.780000,1,2,93.78,4,0.00,55.14,38.64,NaN,0,0,4,0,0,NaN
1,167506448,4,106.320000,1,2,106.32,4,15.97,72.96,33.36,-12.78,0,0,4,2,0,NaN
2,249904235,6,189.500000,1,3,189.50,6,15.90,61.46,128.04,NaN,0,0,6,0,0,NaN
3,23664855,1,19.683529,8,5,185.94,34,0.00,55.86,130.08,-50.00,14,4,30,14,4,32.0
4,3734679,0,16.905000,4,3,67.62,8,0.00,4.32,63.30,0.00,4,0,8,2,0,NaN


In [37]:
df.shape

(100, 17)

In [38]:
q = [
    """
    SELECT 
        COUNT(*)
    FROM 
         dbo.cust_agg
    """
]
df = gp.dmno_execute(q, connection='azure')
print(df)

COMPLETED! 0.18 seconds.
          
0  1146571


### 4. Next step is to add demograpohic data with customer transaction data to crate: cust_demo

In [39]:
query_list = [
    """
    IF OBJECT_ID('dbo.cust_demo', 'U') IS NOT NULL
    DROP TABLE dbo.cust_demo
    """
]
gp.dmno_execute(query_list, connection='azure', result_set=False)

In [40]:
q = [
    """
    CREATE TABLE dbo.cust_demo
    --WITH(DISTRIBUTION=HASH(customer_key)) AS SELECT TOP 100  
    WITH(DISTRIBUTION=HASH(customer_key)) AS 
    SELECT 
        a.*,
        
        b.age,
        b.mega_age,
        b.mega_type_age,
        b.gender,
        b.nm1_gender,
        b.nm1_marital_status,
        b.mega_homeowner,
        b.mega_type_homeowner,
        b.mega_education,
        b.mega_type_education,
        b.ethnic_code,
        b.group_ethnic_code,
        b.mega_presence_of_children,
        b.mega_type_presence_of_children,
        b.mega_number_of_children,
        b.mega_type_number_of_children,
        b.address_type,

        b.mega_income,
        b.mega_type_income,
        b.adjusted_net_worth,
        b.home_value,
        b.credit_ranges,
        b.new_credit_range,
        b.num_credit_lines,
        b.bank_card,
        b.premium_bank_card,
        b.retail_card,
        b.upscale_retail_card,

        b.social_iq_facebook,
        b.social_iq_twitter,
        b.mega_fitness_or_exercise,
        b.mega_type_fitness_or_exercise,
        b.merkle_inf_cardholders_rank,
        b.merkle_marketing_index,
        b.merkle_marketing_rank,
        b.merkle_responder_rating,
        b.spoken_language
                        
    FROM 
        dbo.cust_agg AS a
    
   
    LEFT JOIN
        dbo.ODS_cust_demg_t as b
    ON (a.customer_key = b.cust_key)

    
    """
]
r = gp.dmno_execute(q, connection='azure', result_set=False)


In [41]:
q = [
    """
    SELECT 
        TOP 100 *
    FROM 
         dbo.cust_demo
    """
]
df = gp.dmno_execute(q, connection='azure')

COMPLETED! 0.31 seconds.


In [42]:
df.head()

,customer_key,avg_basket_size,avg_basket_price,count_num_txn,count_num_product,total_sales_amt,count_num_items,min_sales_amt,total_cost_amt,total_diff_sale_cost,total_discount_amt,count_num_return,count_online_ord,count_retail_ord,count_onsale_flag,count_promo_code,avg_daysinto_promo,age,mega_age,mega_type_age,gender,nm1_gender,nm1_marital_status,mega_homeowner,mega_type_homeowner,mega_education,mega_type_education,ethnic_code,group_ethnic_code,mega_presence_of_children,mega_type_presence_of_children,mega_number_of_children,mega_type_number_of_children,address_type,mega_income,mega_type_income,adjusted_net_worth,home_value,credit_ranges,new_credit_range,num_credit_lines,bank_card,premium_bank_card,retail_card,upscale_retail_card,social_iq_facebook,social_iq_twitter,mega_fitness_or_exercise,mega_type_fitness_or_exercise,merkle_inf_cardholders_rank,merkle_marketing_index,merkle_marketing_rank,merkle_responder_rating,spoken_language
0,1626717,0,-85.082353,8,8,141.60,34,0.00,48.96,92.64,7.105427e-15,16,34,0,0,1,18.0,37,50.0,A,F,M,M,Y,A,3.0,A,01,K,Y,A,3.0,A,S,5.0,A,230808.0,U,A,0.0,1.0,Y,None,Y,None,2.0,6.0,0.0,M,7.0,503.0,9.0,1.0,01
1,27667406,3,439.254286,3,3,634.78,14,0.00,209.66,425.12,-1.579800e+02,4,12,2,0,3,48.0,68,68.0,A,F,F,M,Y,A,4.0,A,10,K,Y,A,2.0,A,S,6.0,A,268076.0,M,E,0.0,3.0,Y,None,Y,Y,4.0,2.0,1.0,A,10.0,662.0,10.0,4.0,01
2,6088011,2,28.780000,1,1,28.78,2,14.39,13.34,15.44,-2.880000e+00,0,0,2,0,1,26.0,59,62.0,A,F,M,M,Y,A,3.0,A,01,K,Y,A,4.0,A,S,9.0,A,302625.0,M,B,7.0,1.0,Y,None,Y,Y,2.0,3.0,1.0,A,10.0,677.0,10.0,4.0,01
3,257368650,3,334.466667,2,3,531.40,6,29.70,107.12,424.28,-1.888000e+02,0,2,4,0,1,1.0,0,36.0,A,M,None,M,Y,A,2.0,A,85,A,Y,A,2.0,A,S,6.0,A,162766.0,K,C,0.0,0.0,Y,None,None,None,4.0,5.0,0.0,M,8.0,172.0,6.0,1.0,01
4,1009197,4,432.000000,1,2,432.00,4,68.00,90.92,341.08,-2.160000e+02,0,0,4,0,1,5.0,49,50.0,A,F,F,M,R,A,3.0,A,10,K,Y,A,1.0,A,S,8.0,A,132808.0,M,B,0.0,0.0,Y,Y,Y,None,2.0,6.0,0.0,M,8.0,112.0,5.0,1.0,01


In [43]:
q = [
    """
    SELECT 
        COUNT(*)
    FROM 
         dbo.cust_demo
    """
]
df = gp.dmno_execute(q, connection='azure')
print(df)

COMPLETED! 0.19 seconds.
          
0  1146571


# Finished Data Creation

### Run some exploration on season_desc

In [28]:
q = [
    """
    SELECT 
        DISTINCT(season_desc)
    FROM 
         dbo.cust_trn
    where datalength(season_desc) > 8
    or upper(season_desc) = 'BASIC'
    or upper(season_desc) = 'F1A 2007'
    """
]
df = gp.dmno_execute(q, connection='azure')

COMPLETED! 32.4 seconds.


In [29]:
print(df)
#df.to_csv("Input/distinct_seasondesc.csv")

                season_desc
0                     BASIC
1     SEASONAL BASIC 1 2015
2                BASIC 2013
3     NOV MD @ DECEMBER WK2
4              HOLIDAY 2012
5             MDS Generated
6      JUNE 1 MD @ AUG WK 3
7           HOLIDAY 1A 2014
8   OCTOBER MD@ HOLIDAYSALE
9              DISCONTINUED
10  EARLY AUG MD @ SEPT WK1


In [35]:
q = [
    """
    SELECT 
        season_desc, LEFT(season_desc, 1) as instore_month,
        CAST(SUBSTRING(season_desc, 2,2) as int) as markdown_month,
        CAST(RIGHT(season_desc, 4) as int) as year
    FROM 
         dbo.cust_trn
    WHERE season_desc is not null 
    AND datalength(season_desc) = 8
    and upper(season_desc) <> 'F1A 2007'
   
    """
]
df = gp.dmno_execute(q, connection='azure')

COMPLETED! 514.4 seconds.


In [36]:
df.shape

(5302751, 4)

In [53]:
q = [
    """
    SELECT 
        season_desc, count(*)
    FROM 
         dbo.cust_trn
    where 1 = 1
    and season_desc is null
    or datalength(season_desc) > 8
    or upper(season_desc) in ('F1A 2007', 'BASIC')
    group by season_desc
    """
]
df = gp.dmno_execute(q, connection='azure')

COMPLETED! 0.64 seconds.


In [54]:
df.head(10)

,season_desc,
0,None,1381466
1,BASIC,996220
2,SEASONAL BASIC 1 2015,23
3,F1A 2007,30
4,BASIC 2013,143
5,NOV MD @ DECEMBER WK2,16
6,HOLIDAY 2012,1
7,MDS Generated,2893
8,JUNE 1 MD @ AUG WK 3,66
9,HOLIDAY 1A 2014,4


In [55]:
df.to_csv("Input/nonstandard_seasondesc.csv")

In [6]:
q = [
    """
    with data as
    (
    SELECT TOP 10
        
        season_desc, 
        LEFT(season_desc, 1) as instore_m,
        CASE  
            WHEN LEFT(season_desc, 1) = 'J' THEN 1
            WHEN LEFT(season_desc, 1) = 'F' THEN 2
            WHEN LEFT(season_desc, 1) = 'M' THEN 3
            WHEN LEFT(season_desc, 1) = 'A' THEN 4
            WHEN LEFT(season_desc, 1) = 'Y' THEN 5
            WHEN LEFT(season_desc, 1) = 'U' THEN 6
            WHEN LEFT(season_desc, 1) = 'L' THEN 7
            WHEN LEFT(season_desc, 1) = 'G' THEN 8
            WHEN LEFT(season_desc, 1) = 'S' THEN 9
            WHEN LEFT(season_desc, 1) = 'O' THEN 10
            WHEN LEFT(season_desc, 1) = 'N' THEN 11
            WHEN LEFT(season_desc, 1) = 'D' THEN 12
            ELSE 3 --LEFT(season_desc, 1)
        END as instore_month,
        CAST(SUBSTRING(season_desc, 2,2) as int) as markdown_mm,
        CAST(RIGHT(season_desc, 4) as int) as year
    FROM 
         dbo.cust_trn
    WHERE season_desc is not null 
    AND datalength(season_desc) = 8
    and upper(season_desc) <> 'F1A 2007'
   )
   
   select season_desc, instore_m, instore_month, year, 
       datefromparts(year, instore_month, 1) as instore_date,
       markdown_mm,
       datefromparts(year, markdown_mm, 1) as markdown_date
   from data d
   
    """
]
df = gp.dmno_execute(q, connection='azure')

COMPLETED! 0.32 seconds.


In [7]:
df

,season_desc,instore_m,instore_month,year,instore_date,markdown_mm,markdown_date
0,F05 2017,F,2,2017,2017-02-01,5,2017-05-01
1,M04 2017,M,3,2017,2017-03-01,4,2017-04-01
2,J04 2016,J,1,2016,2016-01-01,4,2016-04-01
3,D03 2016,D,12,2016,2016-12-01,3,2016-03-01
4,M04 2017,M,3,2017,2017-03-01,4,2017-04-01
5,J06 2016,J,1,2016,2016-01-01,6,2016-06-01
6,F03 2017,F,2,2017,2017-02-01,3,2017-03-01
7,N01 2016,N,11,2016,2016-11-01,1,2016-01-01
8,F03 2017,F,2,2017,2017-02-01,3,2017-03-01
9,G03 2016,G,8,2016,2016-08-01,3,2016-03-01


## Try in query

In [5]:
query_list=[
    """
SELECT
       count(*)
        
    FROM 
        dbo.ods_orderline_t AS a
        
    LEFT JOIN 
        dbo.cust_ord_txn_xref_t AS b
    ON a.transaction_num = b.transaction_num
    
    LEFT JOIN
        dbo.ods_orderline_discounts_t as c
    ON (a.transaction_num = c.transaction_num AND a.product_key = c.product_key)
    
    LEFT JOIN 
        dbo.ods_product_t as d
    ON c.product_key = d.product_key
   
    WHERE 
        
        --(CONVERT(date, CAST(txn_dt AS Date), 112) >= '2017-01-29'
        --OR CONVERT(date, CAST(txn_dt AS Date), 112) <= '2018-02-03')
        CONVERT(date, CAST(txn_dt AS Date), 112) BETWEEN '2017-09-01' AND '2017-09-05'
        --txn_dt in ('01-SEP-17', '02-SEP-17', '03-SEP-17', '04-SEP-17', '05-SEP-17')
        AND a.country = 'US'
        AND a.brand = 'BR'
            
    
    """
]

In [6]:
%%time
rr = gp.dmno_execute(query_list, connection='azure')
print(rr)
# 212.67 seconds. - 4060834

COMPLETED! 1107.86 seconds.
          
0  4060834
CPU times: user 534 ms, sys: 235 ms, total: 769 ms
Wall time: 18min 28s


Lets view the table

In [18]:
query_list = [
    """
    CREATE TABLE dbo.cust_ord3_2017
    --WITH(DISTRIBUTION=HASH(customer_key)) AS SELECT TOP 100  
    WITH(DISTRIBUTION=HASH(customer_key)) AS SELECT
        a.customer_key,
        a.transaction_num, 
        a.order_num, 
        a.order_status, 
        a.brand, 
        a.txn_dt, 
        a.loyalty_crdhldr_flag,

        b.line_num, 
        b.order_status as order_status2, 
        b.brand as brand2, 
        b.product_key,
        b.item_type,
        b.item_qty, 
        b.sales_amt,
        
        c.discount_reason_cd,
        c.discount_amt,
        c.discount_seq_nbr,
        c.discount_level,
        c.mkt_pmotn_cd,
        
        d.style_cd,
        d.color_cd,
        d.season_desc,
        d.prod_desc,
        d.vendor_name
        
    FROM 
        dbo.cust_ord_txn_xref_t AS a
        
    RIGHT JOIN 
        dbo.ods_orderline_t AS b
    ON a.transaction_num = b.transaction_num
    
    LEFT JOIN
        dbo.ods_orderline_discounts_t as c
    ON (b.transaction_num = c.transaction_num AND b.product_key = c.product_key)
    
    LEFT JOIN 
        dbo.ods_product_t as d
    ON c.product_key = d.product_key
   
    WHERE 
        a.country = 'US'
        AND a.brand = 'BR'
        --AND YEAR(CONVERT(VARCHAR, CAST(a.txn_dt AS Date), 112)) = '2017'
        AND (CONVERT(date, CAST(txn_dt AS Date), 112) >= '2017-01-29'
            OR CONVERT(date, CAST(txn_dt AS Date), 112) <= '2018-02-03')

        --AND a.txn_dt = '29-JAN-17'
        --AND RIGHT(a.txn_dt, 2) ='17' 
        --AND (RIGHT(a.txn_dt, 2) ='17' OR (SUBSTRING(a.txn_dt, 4, 3)='JAN' AND RIGHT(a.txn_dt, 2) ='18'))    
    """
]

In [19]:
gp.dmno_execute(query_list, connection='azure', result_set=False)

Lets view the table

In [20]:
query_list = [
    """
    SELECT  TOP 100 *
    FROM
        dbo.cust_ord3_2017
        
    ORDER BY
        customer_key, transaction_num, line_num
    
    """
]

In [21]:
result = gp.dmno_execute(query_list, connection='azure')

COMPLETED! 141.09 seconds.


In [5]:
query_list = [
    """
    SELECT total_elapsed_time, command, *
    FROM sys.dm_pdw_exec_requests 
    WHERE status='Running'
    ORDER BY start_time ASC;
    """
]

In [7]:
f=gp.dmno_execute(query_list, connection='azure')

COMPLETED! 0.36 seconds.


In [9]:
f.head()

AttributeError: 'NoneType' object has no attribute 'head'

In [ ]:
KILL 'SESSION_ID'

## Join dm_src and cust_ord_txn_xref_t

In [3]:
query_list = [
    """
    DROP TABLE dbo.cust_ord
    """
]

In [4]:
gp.dmno_execute(query_list, connection='azure', result_set=False)

In [7]:
query_list = [
    """
    CREATE TABLE dbo.cust_ord
    WITH(DISTRIBUTION=HASH(customer_key)) AS SELECT 
        a.fiscal_year, 
        a.fiscal_month, 
        a.recency, 
        a.customer_key, 
        a.country, 
        a.transaction_num,

        b.order_num, 
        b.order_status, 
        b.order_type, 
        b.brand, 
        b.household_key,
        b.first_purchase_brand_flag, 
        b.first_purchase_chan_flag,
        b.item_qty, 
        b.gross_sales_amt, 
        b.loyalty_crdhldr_flag
    FROM 
        dbo.dm_src AS a
    INNER JOIN 
        dbo.cust_ord_txn_xref_t AS b
    ON a.transaction_num = b.transaction_num
    WHERE 
        a.fiscal_year = '2017'
        AND a.fiscal_month = '12'
        AND a.country = 'US'
        AND a.brand = 'BR'
    """
]

In [8]:
gp.dmno_execute(query_list, connection='azure', result_set=False)

In [17]:
query_list = [
    """
    SELECT TOP 100
        *
    FROM
        dbo.cust_ord
    
    """
]
# query_list = [
#     """
#     SELECT COUNT(*)
#     FROM
#         dbo.cust_ord
    
#     """
# ]


In [18]:
result = gp.dmno_execute(query_list, connection='azure')

COMPLETED! 0.24 seconds.


In [19]:
result.head()

,fiscal_year,fiscal_month,recency,customer_key,country,transaction_num,order_num,order_status,order_type,brand,household_key,first_purchase_brand_flag,first_purchase_chan_flag,item_qty,gross_sales_amt,loyalty_crdhldr_flag
0,2017,12,5,197988334,US,2401139809,None,R,None,BR,397971515.0,N,N,3,213.98,None
1,2017,12,5,111328387,US,2387515992,TLQT9MT,O,I,BR,14522041.0,N,N,1,69.99,None
2,2017,12,5,232974571,US,2420060347,None,R,None,BR,400676484.0,N,N,-1,-66.00,None
3,2017,12,5,302484296,US,2535180107,None,R,None,BR,NaN,N,N,-4,-206.70,None
4,2017,12,5,297237790,US,2415513012,None,R,None,BR,NaN,N,N,-1,-82.80,None


In [1]:
query_list = [
    """
    SELECT total_elapsed_time, command, *
FROM sys.dm_pdw_exec_requests 
WHERE status='Running'
ORDER BY start_time ASC;
    
    """
]


In [2]:
df = gp.dmno_execute(query_list, connection='azure')

NameError: name 'gp' is not defined

In [8]:
df

,total_elapsed_time,command,request_id,session_id,status,submit_time,start_time,end_compile_time,end_time,total_elapsed_time,label,error_id,database_id,command,resource_class
0,3891737,"\nIF (NOT EXISTS (SELECT *\n FROM INFORMATION_SCHEMA.TABLES\n WHERE TABLE_SCHEMA = 'dbo'\n AND TABLE_NAME = 'ODS_ORDERLINE_DISCOUNTS_T'))\nBEGIN\nCREATE TABLE dbo.ODS_ORDERLINE_DISCOUNTS_T WITH\n(\n CLUSTERED COLUMNSTORE INDEX,\n DISTRIBUTION = HASH([transaction_num]) \n ,PARTITION([transaction_date] RANGE RIGHT FOR VALUES ('2000-01-01','2001-01-01','2002-01-01','2003-01-01','2004-01-01','2005-01-01','2006-01-01','2007-01-01','2008-01-01','2009-01-01','2010-01-01','2011-01-01','2012-01-01','2013-01-01','2014-01-01','2015-01-01','2016-01-01','2017-01-01','2018-01-01')) \n)\nAS\nSELECT flow_key,transaction_num,line_num,discount_reason_cd,discount_amt,discount_seq_nbr,customer_key,brand,product_key,product_size_desc,non_merch_code,on_sale_flag,sales_amt,promo_cd_key,order_status,discount_level,brand_cd,country,ins_dt,updt_dt,mkt_pmotn_cd,crt_tmst,updt_tmst,epoc_id,transaction_date from dbo.ODS_ORDERLINE_DISCOUNTS_T_INC\n\nEND\nELSE\nBEGIN\nDELETE FROM dbo.ODS_ORDERLINE_DISCOUNTS_T\nWHERE ((epoc_id=1530589634 and transaction_date='2016-06-30') OR (epoc_id=1530589634 and transaction_date='2016-07-01') OR (epoc_id=1530589634 and transaction_date='2017-01-30') OR (epoc_id=1530589634 and transaction_date='2017-12-25') OR (epoc_id=1530594576 and transaction_date='2016-07-02') OR (epoc_id=1530594576 and transaction_date='2016-07-03') OR (epoc_id=1530594576 and transaction_date='2016-07-04') OR (epoc_id=1530594576 and transaction_date='2016-07-05') OR (epoc_id=1530594576 and transaction_date='2016-07-06') OR (epoc_id=1530594576 and transaction_date='2016-07-07') OR (epoc_id=1530594576 and transaction_date='2016-07-08') OR (epoc_id=1530594576 and transaction_date='2016-07-09') OR (epoc_id=1530594576 and transaction_date='2016-07-10') OR (epoc_id=1530594576 and transaction_date='2016-07-11') OR (epoc_id=1530594576 and transaction_date='2016-07-12') OR (epoc_id=1530594576 and transaction_date='2016-07-13') OR (epoc_id=1530594576 and transaction_date='2016-07-14') OR (epoc_id=1530594576 and transaction_date='2016-07-15') OR (epoc_id=1530594576 and transaction_date='2016-07-16') OR (epoc_id=1530594576 and transaction_date='2016-07-17') OR (epoc_id=1530594576 and transaction_date='2016-07-18') OR (epoc_id=1530594576 and transaction_date='2016-07-19') OR (epoc_id=1530594576 and transaction_date='2016-07-20') OR (epoc_id=1530594576 and transaction_date='2016-07-21') OR (epoc_id=1530594576 and transaction_date='2016-07-22') OR (epoc_id=1530594576 and transaction_date='2016-07-23') OR (epoc_id=1530594576 and transaction_date='2016-07-24') OR (epoc_id=1530594576 and transaction_date='2016-07-25') OR (epoc_id=1530594576 and transaction_date='2016-07-26') OR (epoc_id=1530594576 and transaction_date='2016-07-27') OR (epoc_id=1530594576 and transaction_date='2016-07-28') OR (epoc_id=1530594576 and transaction_date='2016-07-29') OR (epoc_id=1530594576 and transaction_date='2016-07-30') OR (epoc_id=1530594576 and transaction_date='2016-07-31') OR (epoc_id=1530594576 and transaction_date='2016-08-01') OR (epoc_id=1530594576 and transaction_date='2016-08-02') OR (epoc_id=1530594576 and transaction_date='2016-08-03') OR (epoc_id=1530594576 and transaction_date='2016-08-04') OR (epoc_id=1530594576 and transaction_date='2016-08-05') OR (epoc_id=1530594576 and transaction_date='2016-08-06') OR (epoc_id=1530594576 and transaction_date='2016-08-07') OR (epoc_id=1530594576 and transaction_date='2016-08-08') OR (epoc_id=1530594576 and transaction_date='2016-08-09') OR (epoc_id=1530594576 and transaction_date='2016-08-10') OR (epoc_id=1530594576 and transaction_date='2016-08-11') OR (epoc_id=1530594576 and transaction_date='2016-08-12') OR (epoc_id=1530594576 and transaction_date='2016-08-13') OR (epoc_id=1530594576 and transaction_date='2016-08-14') OR (epoc_id=1530594576 and trans

In [ ]:
KILL 'SESSION_ID'